Download dl data and save it in the system as csv.   
check MX, IN for the edges before 0431, 0731, cause some data might be missing, need to add them 

In [1]:
import gc
import os
import time
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime, timedelta

In [2]:
# SQL engine
from trino.dbapi import connect 
from sqlalchemy import create_engine
import pandas as pd
import time

class TrinoEngine():
    def __init__(self):
        conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query:str) -> list:
        """
        Create and drop statements.
        """
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql(self, query:str) -> pd.DataFrame: 
        """
        Select and insert into operations.
        """
        return pd.read_sql(query, self.engine)

sql_engine = TrinoEngine()

In [3]:
schema_name = {'cda': 'cuebiq.paas_cda_pe_v3'}

# dl_table = f"{schema_name['cda']}.device_location"  
pe_dl_table = f"{schema_name['cda']}.device_location_uplevelled"

tj_table = f"{schema_name['cda']}.trajectory"     
pe_tj_table = f"{schema_name['cda']}.trajectory_uplevelled"

# stop_table = f"{schema['cda']}.stop" 
pe_stop_table = f"{schema_name['cda']}.stop_uplevelled"

visit_table = f"{schema_name['cda']}.visit " 

In [4]:
class TrinoEngine:
    def __init__(self):
        self.conn = connect(
            host="localhost",
            port=9090,
            catalog="cuebiq"
        )
        self.cur = self.conn.cursor()
        self.engine = create_engine("trino://localhost:9090/cuebiq/")
    
    def execute_statement(self, query: str):
        self.cur.execute(query)
        return self.cur.fetchall()
    
    def read_sql_chunked(self, query: str, chunksize: int = 10000):
        return pd.read_sql(query, self.engine, chunksize=chunksize)

sql_engine = TrinoEngine()


# dl

In [10]:
# Add event_datetime_local to pre downloaded

In [1]:
import os
import pandas as pd
from datetime import datetime

# Function to parse the `event_zoned_datetime` and generate `event_datetime_local`
def generate_event_datetime_local(df):
    df['event_datetime_local'] = pd.to_datetime(df['event_zoned_datetime'].str[:19], format='%Y-%m-%dT%H:%M:%S')
    return df

In [8]:
# Path to the folder containing CSV files
folder_path = '/home/jovyan/Data/DL/MX/'

In [9]:
# List all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Chunk size for reading and processing
chunk_size = 10000

# Iterate through each CSV file and process it in chunks
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Initialize an empty list to store chunks
    chunks = []

    # Read the CSV file in chunks
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        # Generate `event_datetime_local` column for each chunk
        chunk = generate_event_datetime_local(chunk)
        chunks.append(chunk)
    
    # Concatenate all chunks into a single DataFrame
    df = pd.concat(chunks, ignore_index=True)
    
    # Save the processed DataFrame back to the same CSV file (overwrite)
    df.to_csv(file_path, index=False)
    
    # Print a message to indicate processing is done
    print(f"Processed and overwritten {csv_file}")

Processed and overwritten 20190111_MX_pe_dl.csv
Processed and overwritten 20190102_MX_pe_dl.csv
Processed and overwritten 20190120_MX_pe_dl.csv
Processed and overwritten 20190119_MX_pe_dl.csv


KeyboardInterrupt: 

In [12]:
# Select based on event_datetime_local direcly but very slow

countries = ['CO']
start_date = 20190101
end_date = 20190103

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Start timing
start_time = time.time()

for country_code in countries:
    print(f"Processing country: {country_code}")
    current_date = start_date_dt
    while current_date <= end_date_dt:
        formatted_date = current_date.strftime('%Y%m%d')
        next_date = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        print(f"Processing date: {formatted_date} for country: {country_code}")

        query = f"""
            WITH event_data AS (
                SELECT 
                    cuebiq_id, 
                    event_zoned_datetime, 
                    processing_date,
                    timezoneoffset_secs,
                    lat,
                    lng, 
                    TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
                FROM {pe_dl_table}
                WHERE 
                    country_code = '{country_code}' 
                    AND event_zoned_datetime IS NOT NULL
                    AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
            )
            SELECT 
                cuebiq_id, 
                event_zoned_datetime, 
                processing_date,
                timezoneoffset_secs,
                lat,
                lng
            FROM event_data
            WHERE 
                event_datetime_local >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
                AND event_datetime_local < date_parse('{next_date}', '%Y-%m-%d')
        """

        for chunk in sql_engine.read_sql_chunked(query):
            # Extract event_zoned_date from event_zoned_datetime
            chunk['event_zoned_date'] = chunk['event_zoned_datetime'].apply(lambda x: x[:10])

            # Convert processing_date and event_zoned_date to datetime objects
            chunk['event_zoned_date'] = pd.to_datetime(chunk['event_zoned_date'])
            processing_date_dt = datetime.strptime(formatted_date, '%Y%m%d')

            # Filter data based on event_zoned_date
            chunk = chunk[(chunk['event_zoned_date'] >= start_date_dt.strftime('%Y-%m-%d')) & 
                          (chunk['event_zoned_date'] <= processing_date_dt.strftime('%Y-%m-%d'))]

            # Group by event_zoned_date
            grouped = chunk.groupby('event_zoned_date')

            for event_zoned_date, group_df in grouped:
                # Remove duplicates
                group_df = group_df.drop_duplicates()

                # Convert event_zoned_date to string and remove hyphens
                event_zoned_date_str = event_zoned_date.strftime('%Y%m%d')
                file_path = f'/home/jovyan/Data/DL/{country_code}/{event_zoned_date_str}_{country_code}_pe_dl.csv'

                if os.path.exists(file_path):
                    # If the file exists, append the new data
                    group_df.to_csv(file_path, mode='a', header=False, index=False)
                else:
                    # If the file does not exist, create it
                    try:
                        group_df.to_csv(file_path, index=False)
                    except OSError as e:
                        if not os.path.exists(os.path.dirname(file_path)):
                            os.makedirs(os.path.dirname(file_path), exist_ok=True)
                            group_df.to_csv(file_path, index=False)
                        else:
                            raise e

            # Delete the DataFrame to free up memory
            del chunk

        current_date += timedelta(days=1)

    print(f"Finished processing country: {country_code}")

# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


# Processing date: 20190114 for country: IN
# Processing date: 20190430 for country: ID
# Processing date: 20190130 for country: MX

# Works but Slowwwwwww!  7min for 3 day co

Processing country: CO
Processing date: 20190101 for country: CO
Processing date: 20190102 for country: CO
Processing date: 20190103 for country: CO
Finished processing country: CO
Total time taken: 422.44857931137085 seconds


In [7]:
# Generate event_datetime_local column and export
# Using This one!!!!!!!!!!!!!!!!!!! (I think)

countries = ['ID']
start_date = 20190801
end_date = 20190831

# Convert integer dates to datetime objects
start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# Start timing
start_time = time.time()

for country_code in countries:
    print(f"Processing country: {country_code}")
    current_date = start_date_dt
    while current_date <= end_date_dt:
        formatted_date = current_date.strftime('%Y%m%d')
        next_date = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        print(f"Processing date: {formatted_date} for country: {country_code}")

        query = f"""
            SELECT 
                cuebiq_id, 
                event_zoned_datetime, 
                processing_date,
                lat,
                lng, 
                TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local

            FROM {pe_dl_table}
            WHERE 
                processing_date = {formatted_date} 
                AND country_code = '{country_code}' 
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('20190101', '%Y%m%d')
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
                AND event_zoned_datetime IS NOT NULL
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        """

        for chunk in sql_engine.read_sql_chunked(query):
            # Extract event_zoned_date from event_zoned_datetime
            chunk['event_zoned_date'] = chunk['event_zoned_datetime'].apply(lambda x: x[:10])

            # Convert processing_date and event_zoned_date to datetime objects
            chunk['event_zoned_date'] = pd.to_datetime(chunk['event_zoned_date'])
            processing_date_dt = datetime.strptime(formatted_date, '%Y%m%d')

            # Filter data based on event_zoned_date
            chunk = chunk[(chunk['event_zoned_date'] >= start_date_dt.strftime('%Y-%m-%d')) & 
                          (chunk['event_zoned_date'] <= processing_date_dt.strftime('%Y-%m-%d'))]

            # Group by event_zoned_date
            grouped = chunk.groupby('event_zoned_date')

            for event_zoned_date, group_df in grouped:
                # Remove duplicates within the current group_df
                group_df = group_df.drop_duplicates()

                # Convert event_zoned_date to string and remove hyphens
                event_zoned_date_str = event_zoned_date.strftime('%Y%m%d')
                file_path = f'/home/jovyan/Data/DL/{country_code}/{event_zoned_date_str}_{country_code}_pe_dl.csv'

                if os.path.exists(file_path):
                    # If the file exists, read the existing data
                    existing_df = pd.read_csv(file_path)

                    # Concatenate the existing data with the new group_df
                    combined_df = pd.concat([existing_df, group_df])

                    # Remove duplicates from the combined DataFrame
                    combined_df = combined_df.drop_duplicates()

                    # Save the combined DataFrame back to the file
                    combined_df.to_csv(file_path, index=False)
                else:
                    # If the file does not exist, create it
                    try:
                        group_df.to_csv(file_path, index=False)
                    except OSError as e:
                        if not os.path.exists(os.path.dirname(file_path)):
                            os.makedirs(os.path.dirname(file_path), exist_ok=True)
                            group_df.to_csv(file_path, index=False)
                        else:
                            raise e

            # Delete the DataFrame to free up memory
            del chunk

        current_date += timedelta(days=1)

    print(f"Finished processing country: {country_code}")

# End timing
end_time = time.time()

# Calculate and print the total time taken
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


# Processing date: 20190131 for country: IN
# Processing date: 20190731 for country: ID   1 hour for 10 day
# Processing date: 20190228 for country: MX   

Processing country: ID
Processing date: 20190801 for country: ID


KeyboardInterrupt: 

In [ ]:
import os
import time
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine
from pandas.errors import EmptyDataError

# Function to process data for a given date range and country
def process_country_data(country_code, start_date_dt, end_date_dt):
    current_date = start_date_dt
    while current_date <= end_date_dt:
        formatted_date = current_date.strftime('%Y%m%d')
        next_date = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
        print(f"Processing date: {formatted_date} for country: {country_code}")

        query = f"""
            SELECT 
                cuebiq_id, 
                event_zoned_datetime, 
                processing_date,
                lat,
                lng, 
                TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) AS event_datetime_local
            FROM {pe_dl_table}
            WHERE 
                processing_date = {formatted_date} 
                AND country_code = '{country_code}' 
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('20190101', '%Y%m%d')
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
                AND event_zoned_datetime IS NOT NULL
                AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
        """

        batch_data = []

        for chunk in sql_engine.read_sql_chunked(query):
            chunk['event_zoned_date'] = pd.to_datetime(chunk['event_zoned_datetime'].str[:10])

            processing_date_dt = datetime.strptime(formatted_date, '%Y%m%d')
            chunk = chunk[(chunk['event_zoned_date'] >= start_date_dt) & 
                          (chunk['event_zoned_date'] <= processing_date_dt)]

            grouped = chunk.groupby('event_zoned_date')
            for event_zoned_date, group_df in grouped:
                group_df = group_df.drop_duplicates()
                event_zoned_date_str = event_zoned_date.strftime('%Y%m%d')
                batch_data.append((event_zoned_date_str, group_df))

            del chunk

        # Save batch data
        for event_zoned_date_str, group_df in batch_data:
            file_path = f'/home/jovyan/Data/DL/{country_code}/{event_zoned_date_str}_{country_code}_pe_dl.csv'
            if os.path.exists(file_path):
                try:
                    existing_df = pd.read_csv(file_path)
                    combined_df = pd.concat([existing_df, group_df]).drop_duplicates()
                except EmptyDataError:
                    combined_df = group_df
                combined_df.to_csv(file_path, index=False)
            else:
                os.makedirs(os.path.dirname(file_path), exist_ok=True)
                group_df.to_csv(file_path, index=False)

        current_date += timedelta(days=1)
    print(f"Finished processing country: {country_code}")

# Main script
countries = ['ID']
start_date = 20190801
end_date = 20190831

start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

start_time = time.time()

for country_code in countries:
    process_country_data(country_code, start_date_dt, end_date_dt)

end_time = time.time()
total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")


Processing date: 20190801 for country: ID


In [5]:
df = pd.read_csv('/home/jovyan/Data/DL/ID/20190613_ID_pe_dl.csv')
df.sort_values('event_datetime_local')

,cuebiq_id,event_zoned_datetime,processing_date,lat,lng,event_datetime_local,event_zoned_date
1318392,1972801775,2019-06-13T00:00:00+07:00,20190613,-5.379151,105.216373,2019-06-13 00:00:00,2019-06-13
4090119,1874892836,2019-06-13T00:00:00+07:00,20190613,1.185788,104.011441,2019-06-13 00:00:00,2019-06-13
3495442,1893971178,2019-06-13T00:00:00+07:00,20190613,-8.513576,114.111792,2019-06-13 00:00:00,2019-06-13
1958164,2089337707,2019-06-13T00:00:00+07:00,20190613,-6.150520,106.869127,2019-06-13 00:00:00,2019-06-13
627766,1980479954,2019-06-13T00:00:00+07:00,20190613,-7.421599,109.478756,2019-06-13 00:00:00,2019-06-13
...,...,...,...,...,...,...,...
2356614,2087388010,2019-06-13T23:59:59+09:00,20190613,-1.179000,136.188175,2019-06-13 23:59:59,2019-06-13
393475,1988646522,2019-06-13T23:59:59+08:00,20190613,-8.668958,115.182401,2019-06-13 23:59:59,2019-06-13
76350,2096776848,2019-06-13T23:59:59+08:00,20190613,-8.167199,114.733030,2019-06-13 23:59:59,2019-06-13
4125366,2065007831,2019-06-13T23:59:59+07:00,20190613,-6.241934,106.963437,2019-06-13 23:59:59,2019-06-13


In [16]:
# # With UTC correction

# countries = ['MX']
# start_date = 20190205
# end_date = 20190205

# # Convert integer dates to datetime objects
# start_date_dt = datetime.strptime(str(start_date), '%Y%m%d')
# end_date_dt = datetime.strptime(str(end_date), '%Y%m%d')

# # Start timing
# start_time = time.time()

# for country_code in countries:
#     print(f"Processing country: {country_code}")
#     current_date = start_date_dt
#     while current_date <= end_date_dt:
#         formatted_date = current_date.strftime('%Y%m%d')
#         next_date = (current_date + timedelta(days=1)).strftime('%Y-%m-%d')
#         print(f"Processing date: {formatted_date} for country: {country_code}")

#         query = f"""
#             SELECT 
#                 cuebiq_id, 
#                 event_zoned_datetime, 
#                 processing_date,
#                 timezoneoffset_secs,
#                 lat,
#                 lng, 
#                 TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s') +
#                 interval '1' second * timezoneoffset_secs) AS event_datetime_utc

#             FROM {pe_dl_table}
#             WHERE 
#                 processing_date = {formatted_date} 
#                 AND country_code = '{country_code}' 
#                 AND event_zoned_datetime IS NOT NULL
#                 AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) IS NOT NULL
#                 AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) >= date_parse('{start_date_dt.strftime('%Y%m%d')}', '%Y%m%d')
#                 AND TRY(date_parse(substr(event_zoned_datetime, 1, 19), '%Y-%m-%dT%H:%i:%s')) <= date_parse('{next_date}', '%Y-%m-%d')
#         """

#         for chunk in sql_engine.read_sql_chunked(query):
#             # Extract event_zoned_date from event_zoned_datetime
#             chunk['event_zoned_date'] = chunk['event_zoned_datetime'].apply(lambda x: x[:10])

#             # Convert processing_date and event_zoned_date to datetime objects
#             chunk['event_zoned_date'] = pd.to_datetime(chunk['event_zoned_date'])
#             processing_date_dt = datetime.strptime(formatted_date, '%Y%m%d')

#             # Filter data based on event_zoned_date
#             chunk = chunk[(chunk['event_zoned_date'] >= start_date_dt.strftime('%Y-%m-%d')) & 
#                           (chunk['event_zoned_date'] <= processing_date_dt.strftime('%Y-%m-%d'))]

#             # Group by event_zoned_date
#             grouped = chunk.groupby('event_zoned_date')

#             for event_zoned_date, group_df in grouped:
#                 # Remove duplicates
#                 group_df = group_df.drop_duplicates()

#                 # Convert event_zoned_date to string and remove hyphens
#                 event_zoned_date_str = event_zoned_date.strftime('%Y%m%d')
#                 file_path = f'/home/jovyan/Data/DL/{country_code}/{event_zoned_date_str}_{country_code}_pe_tj.csv'

#                 if os.path.exists(file_path):
#                     # If the file exists, append the new data
#                     group_df.to_csv(file_path, mode='a', header=False, index=False)
#                 else:
#                     # If the file does not exist, create it
#                     try:
#                         group_df.to_csv(file_path, index=False)
#                     except OSError as e:
#                         if not os.path.exists(os.path.dirname(file_path)):
#                             os.makedirs(os.path.dirname(file_path), exist_ok=True)
#                             group_df.to_csv(file_path, index=False)
#                         else:
#                             raise e

#             # Delete the DataFrame to free up memory
#             del chunk

#         current_date += timedelta(days=1)

#     print(f"Finished processing country: {country_code}")

# # End timing
# end_time = time.time()

# # Calculate and print the total time taken
# total_time = end_time - start_time
# print(f"Total time taken: {total_time} seconds")

# # CO
# # Processing date: 20190114 for country: IN
# # Processing date: 20190430 for country: ID
# # Processing date: 20190130 for country: MX